## Preprocesamiento - Titanic (Kaggle)

### Objetivos
* Imputar o eliminar valores faltantes según criterio justificado.
* Codificar variables categóricas (OneHot, LabelEncoder, etc.).
* Normalizar o estandarizar variables numéricas.
* Analizar si aplicar PCA u otra técnica de reducción de dimensión mejora el modelo.

In [1]:
#Importación de librerías

#%pip install pandas numpy matplotlib seaborn scipy   

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

%matplotlib inline
sns.set_theme(style="darkgrid")

In [2]:
# Carga del dataset de entrenamiento
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Limpieza de datos - Eliminación de columnas

Name: El nombre de la persona no aporta información sobre la posibilidad de supervivencia. Sin embargo dado que aparece el título de la persona (Mr., Ms. Rev., etc) se podría tratar de encontrar una relación entre este tratamiento y el nivel de supervivencia. 

Cabin: Igualmente para el número de cabina en el que se alojaban, sin embargo, se tienen demasiados valores nulos.

Ticket: Dado que no se sabe si el ticket correspondía alguna ubicación determinada en el barco, este campo tampoco aporta información útil

Por esto a fin de simplificar el trabajo se considerará que las columnas PassengerId, Name, Ticket y Cabin no influyen en la supervivencia del pasajero y son irrelevantes a los fines de este trabajo.


In [8]:
modified_df = df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])

modified_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


#### Imputación de datos faltantes

**Age**<br/>Para los datos faltantes de edad (Age) se utilizará el método de imputación simple Hot-Deck ó KNN dado que:

* Usa toda la información disponible del pasajero.
* Mantiene la correlación entre edad y otras variables (clase, sexo, tarifa, etc.).
* Produce valores coherentes con cada individuo.

**Embarked**<br/>Para los datos faltantes de Embarked se utilizará la imputación por la mediana dado que:
* Embarked es una variable categórica con solo 3 categorías.
* Solo hay 2 valores faltantes en el dataset original.
* La distribución es muy desbalanceada: S ~ 72%, C ~ 19%, Q ~ 9%


Edad (Age) - Imputación simple KNN

In [9]:
from sklearn.impute import KNNImputer

CARACTERISTICAS_A_IMPUTAR = ["Age", "Pclass", "SibSp", "Parch", "Fare"]

def impute_age_with_knn(dataframe, n_neighbors=5):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    features = dataframe[CARACTERISTICAS_A_IMPUTAR]
    imputed_array = imputer.fit_transform(features)
    dataframe = dataframe.copy()
    dataframe["Age"] = imputed_array[:, 0]
    return dataframe

completed_df = impute_age_with_knn(modified_df)

Puerto de embarque (Embarked) - Imputación por mediana 

In [10]:
completed_df = completed_df.copy()
completed_df["Embarked"] = completed_df["Embarked"].fillna(completed_df["Embarked"].mode().iloc[0])

completed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


A continuación de muestra una comparación entre las estadísiticas descriptivas edad originales e imputada.

In [11]:
print("\nEstadísticas descriptivas de los datos originales de edad:")
modified_df["Age"].describe()


Estadísticas descriptivas de los datos originales de edad:


count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [12]:
print("\nEstadísticas descriptivas de los datos completados de edad:")
completed_df["Age"].describe()


Estadísticas descriptivas de los datos completados de edad:


count    891.000000
mean      29.863791
std       13.389283
min        0.420000
25%       22.000000
50%       29.000000
75%       37.000000
max       80.000000
Name: Age, dtype: float64

#### Valores atípicos (Outliers)
Si bien el dataset presenta valores atípicos en las columnas edad y tarifa, como se pueda apreciar en la sección de Análisis Exploratorio de Datos, estos no parecen representar errores de registro sino valores extremos, sino que proporcionan información valiosa.<br/>Es por esto que **NO** se eliminarán ni realizarán cambios.


### Codificación de las variables categóricas

### Escalado